In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()


# print("Loaded Environment Variables:", dict(os.environ))

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# print(GOOGLE_API_KEY)


YOUTUBE_VIDEO="https://www.youtube.com/watch?v=cdiD-9MMpb0"


Loaded Environment Variables: {'COMMAND_MODE': 'unix2003', 'HOME': '/Users/srinivasareddymedam', 'HOMEBREW_CELLAR': '/opt/homebrew/Cellar', 'HOMEBREW_PREFIX': '/opt/homebrew', 'HOMEBREW_REPOSITORY': '/opt/homebrew', 'INFOPATH': '/opt/homebrew/share/info:/opt/homebrew/share/info:', 'LOGNAME': 'srinivasareddymedam', 'MallocNanoZone': '0', 'NVM_BIN': '/Users/srinivasareddymedam/.nvm/versions/node/v22.13.1/bin', 'NVM_CD_FLAGS': '-q', 'NVM_DIR': '/Users/srinivasareddymedam/.nvm', 'NVM_INC': '/Users/srinivasareddymedam/.nvm/versions/node/v22.13.1/include/node', 'OLDPWD': '/', 'ORIGINAL_XDG_CURRENT_DESKTOP': 'undefined', 'PATH': '/Users/srinivasareddymedam/Desktop/FirstRAG/.venv/bin:/Users/srinivasareddymedam/google-cloud-sdk/bin:/Users/srinivasareddymedam/google-cloud-sdk/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/Users/srinivasareddymedam/.local/bin:/Users/srinivasareddymedam/.nvm/versions/node/v22.13.1/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/c

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

model="gemini-1.5-pro"
llm=ChatGoogleGenerativeAI(model=model, google_api_key=GOOGLE_API_KEY)


In [3]:
llm.invoke("What MLB team Won the world series during the COVID-19 pandamic?")

AIMessage(content='The Los Angeles Dodgers won the 2020 World Series, which was played during the COVID-19 pandemic.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-0568d81a-6862-44ae-8a49-80a6bd2f3308-0', usage_metadata={'input_tokens': 16, 'output_tokens': 26, 'total_tokens': 42, 'input_token_details': {'cache_read': 0}})

In [ ]:
result=llm.invoke("what is 2+2?")
# print(result.content)

2 + 2 = 4


In [9]:
from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()

chain = llm | parser

chain.invoke("What MLB team Won the world series during the COVID-19 pandamic?")

'The Los Angeles Dodgers won the 2020 World Series, which was played during the COVID-19 pandemic.'

In [10]:
from langchain_core.prompts import ChatPromptTemplate


template="""

Answer the question based on the context below. If you cannot find the answer in the context,
 just say "I don't know". Don't try to make up an answer.

 context: {context}

 question: {question}



"""

prompt=ChatPromptTemplate.from_template(template)
prompt.format(context="Mary's sister is sussana.", question="who is Mary's sister's ?")


'Human: \n\nAnswer the question based on the context below. If you cannot find the answer in the context,\n just say "I don\'t know". Don\'t try to make up an answer.\n\n context: Mary\'s sister is sussana.\n\n question: who is Mary\'s sister\'s ?\n\n\n\n'

In [11]:
chain= prompt | llm | parser

chain.invoke({
    "context":"Mary's sister is sussana.", 
    "question":"who is Mary's sister's ?"
})

'Susanna'

# COMBINING CHAINS IT IS APART FROM THE FLOW 

we can combine different chains to create more complex workflows. for explample lets create a second chain that trans;ltes the result  from first chain into differenbt language 

![Alt Text](Screenshot 2025-03-11 at 23.09.25.png)

In [12]:
translate_prompt=ChatPromptTemplate.from_template(
    "translate {answer} to {language} "
)

In [16]:
from operator import itemgetter

translate_chain=(
    {"answer":chain, "language":itemgetter("Language")}
    | translate_prompt
    | llm
    | parser
)

translate_chain.invoke(
    {
    "context": "Mary's sister is Susana. She doesn't have any more siblings.",
    "question": "How many sisters does Mary have?",
    "Language": "Hindi",
    }
)

'The Hindi translation of "One" is **एक** (ek).'

# Transcribing the youtube video 

the context we want to send to the model comes from a youtube video and transcribe it usning 

In [ ]:
import tempfile
import whisper
from pytube import YouTube


# Let's do this only if we haven't created the transcription file yet.
if not os.path.exists("transcription.txt"):
    youtube = YouTube(YOUTUBE_VIDEO)
    audio = youtube.streams.filter(only_audio=True).first()

    # Let's load the base model. This is not the most accurate
    # model but it's fast.
    whisper_model = whisper.load_model("base")

    with tempfile.TemporaryDirectory() as tmpdir:
        file = audio.download(output_path=tmpdir)
        transcription = whisper_model.transcribe(file, fp16=False)["text"].strip()

        with open("transcription.txt", "w") as file:
            file.write(transcription)